In [ ]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

GREEKS_PATH = Path('data_/greeks.csv')
SAMPLE_SUBMISSION_PATH = Path('data_/sample_submission.csv')
TEST_PATH = Path('data_/test.csv')
TRAIN_PATH = Path('data_/train.csv')
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)
greeks = pd.read_csv(GREEKS_PATH)
FEATURES = [col for col in train.columns if col != 'Id' and col != 'Class']
train['EJ'] = train['EJ'].map(dict((v, k) for k, v in enumerate(train['EJ'].unique())))
test['EJ'] = test['EJ'].map(dict((v, k) for k, v in enumerate(test['EJ'].unique())))
train_dataset = lgb.Dataset(train.drop(['Id', 'Class'], axis=1), train['Class'],
                            feature_name=train.drop(['Id', 'Class'], axis=1).columns.tolist())

x = train.loc[:, FEATURES].values
x = StandardScaler().fit_transform(x)

x_ = test.loc[:, FEATURES].values
x_ = StandardScaler().fit_transform(x_)
train_normalized = pd.DataFrame(x, columns=FEATURES)
train_normalized.fillna(train_normalized.mean(), inplace=True)

test_normalized = pd.DataFrame(x_, columns=FEATURES)
test_normalized.fillna(test_normalized.mean(), inplace=True)
pca = PCA(n_components=2)

train_normalized_pca = pca.fit_transform(train_normalized)
test_normalized_pca = pca.fit_transform(test_normalized)
plt.scatter(train_normalized_pca[0], train_normalized_pca[1], colors=train['Class'])